# The Gladius Data Model

### Data Model Structure


Collection: "clubs"

Document: "club_id"
Club data fields
Sub-collection: "events"
Document: "event_id"
Event data fields
Sub-collection: "occurrences"
Document: "occurrence_id"
Occurrence data fields
Sub-collection: "attendance"
Document: "attendance_id"
Field: "user_id" (to reference the attending user)
Field: "score" (record the score for the user at this occurrence)

Collection: "users"

Document: "user_id"
User data fields
Sub-collection: "scores"
Document: "score_id"
Field: "club_id" (to reference the club)
Field: "event_id" (to reference the event)
Field: "occurrence_id" (to reference the occurrence)
Field: "score" (the user's score for this event occurrence)


In [27]:

# Import firebase_admin and initialize the app
import firebase_admin
from firebase_admin import auth, credentials, firestore



### Populate clubs data model
Add one club

In [25]:

if not firebase_admin._apps:
    cred = credentials.Certificate("firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred
                                                
)
# Get a reference to the firestore database
db = firestore.client()

# Create a collection for clubs
clubs = db.collection("clubs")

# Create a document for a club with some data fields
club_id = "123"
club_data = {
    "name": "Gladius Club",
    "description": "A Learn to Earn Club",
    "members":  ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Harry", "Ivy", "Jack"]
}
club_ref = clubs.document(club_id)
club_ref.set(club_data)

# Create a sub-collection for events under the club document
events = club_ref.collection("events")

# Create a document for an event with some data fields
event_id = "456"
event_data = {
    "title": "Fencing Tournament",
    "date": "2023-06-30",
    "location": "Gladius Arena"
}
event_ref = events.document(event_id)
event_ref.set(event_data)

# Create a sub-collection for occurrences under the event document
occurrences = event_ref.collection("occurrences")

# Create a document for an occurrence with some data fields
occurrence_id = "789"
occurrence_data = {
    "start_time": "14:00",
    "end_time": "16:00",
    "participants": ["Alice", "Bob"]
}
occurrence_ref = occurrences.document(occurrence_id)
occurrence_ref.set(occurrence_data)

# Create a sub-collection for attendance under the occurrence document
attendance = occurrence_ref.collection("attendance")

# Create a document for an attendance with some data fields
attendance_id = "1011"
attendance_data = {
    "name": "Alice",
    "score": 3
}
attendance_ref = attendance.document(attendance_id)
attendance_ref.set(attendance_data)

update_time {
  seconds: 1687176054
  nanos: 929896000
}

### Populate users data

Add one user


In [ ]:

if not firebase_admin._apps:
    cred = credentials.Certificate("firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred
)


# Get a reference to the firestore database
db = firestore.client()

# Create a collection for users
users = db.collection("users")

# Create a document for a user with some data fields
user_id = "Bob"
user_data = {
    "name": "Bob",
    "email": "Bob@bob.com",
    "age": 25
}
user_ref = users.document(user_id)
user_ref.set(user_data)

# Create a sub-collection for scores under the user document
scores = user_ref.collection("scores")

# Create a document for a score with some data fields
score_id = "1213"
score_data = {
    "club_id": "123",
    "event_id": "456",
    "occurrence_id": "789",
    "score": 3
}
score_ref = scores.document(score_id)
score_ref.set(score_data)


update_time {
  seconds: 1686947324
  nanos: 409355000
}

Delete users collection

In [ ]:
# NOT DOING ANYTHING
# Get a reference to the collection of users
users = db.collection("users")

# Delete the collection of users by deleting each document in a batch
batch = db.batch()
for doc in users.stream():
    batch.delete(doc.reference)
batch.commit()

[]

### Import fake users data

In [ ]:
names = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Harry", "Ivy", "Jack"]
users = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Harry", "Ivy", "Jack"]

create auth records

In [3]:
import firebase_admin
from firebase_admin import auth, credentials
#from firebase_admin import credentials, firestore

# Initialize the app with a service account
if not firebase_admin._apps:
    cred = credentials.Certificate("firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)

# Create a list of users to add
users = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Harry", "Ivy", "Jack"]

# Loop through the users and create them with a default password
for user in users:
  try:
    # Create the user with the email and password
    user_record = auth.create_user(
      email=f'{user.lower()}@example.com',
      password='123456'
    )
    # Print the user ID
    print(f'Successfully created new user: {user_record.uid}')
  except firebase_admin.auth.AuthError as e:
    # Handle any errors
    print(f'Error creating new user: {e}')

Successfully created new user: LNy2LVNwsqRDrjG5Qgihi8Vx9bD2
Successfully created new user: 6FbuLmhgD3O3wWAs9TsA6TCgPWr2
Successfully created new user: iVYlpgBdsZet6DJDbEsCUquvT9v1
Successfully created new user: 1keZrqsbPBMZtFhA2Ps2lSYJKaE2
Successfully created new user: wQB08TuN8kQZc9IFONeKSytcyvI2


Populate users data model with scores 

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import random

# Initialize the app with a service account and the database URL
if not firebase_admin._apps:
    cred = credentials.Certificate("firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)

# Get a reference to the firestore database
db = firestore.client()

# Get a reference to the users collection
users = db.collection("users")

# Generate some random user data
names = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Harry", "Ivy", "Jack"]
ages = [random.randint(18, 40) for _ in range(10)]
emails = [name.lower() + "@example.com" for name in names]

# Loop through the user data and set them under a unique document ID
for i in range(10):
    user_id = names[i] # Use the name as the document ID
    user_data = {
        "name": names[i],
        "email": emails[i],
        "age": ages[i]
    }
    user_ref = users.document(user_id)
    user_ref.set(user_data)

    # Create a sub-collection for scores under the user document
    scores = user_ref.collection("scores")

    # Create a document for a score with some data fields
    score_id = str(random.randint(1000, 9999)) # Generate a random score ID
    score_data = {
        "club_id": str(random.randint(100, 999)), # Generate a random club ID
        "event_id": str(random.randint(100, 999)), # Generate a random event ID
        "occurrence_id": str(random.randint(100, 999)), # Generate a random occurrence ID
        "score": random.randint(1, 5) # Generate a random score between 1 and 5
    }
    score_ref = scores.document(score_id)
    score_ref.set(score_data)


Import UID from Auth to users collection

In [15]:
import firebase_admin
from firebase_admin import auth, credentials, firestore


# Initialize the app with a service account
if not firebase_admin._apps:
    cred = credentials.Certificate("firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)

# Get a reference to the Firestore client
db = firestore.client()

# Get a reference to the users collection
users_ref = db.collection('users')

# Create a list of users to query by email
users = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Harry", "Ivy", "Jack"]

# Loop through the users and query their records by email
for user in users:
  try:
    # Get the user record by email
    user_record = auth.get_user_by_email(f'{user.lower()}@example.com')
    # Get the user data from the record
    user_data = {
      'uid': user_record.uid,
      'email': user_record.email,
      'name': user,
      #'display_name': user_record.display_name,
      #'photo_url': user_record.photo_url,
      #'disabled': user_record.disabled
    }
    # Save the user data to the root level collection
    users_ref.document(user).set(user_data)
  except firebase_admin.auth.AuthError as e:
    # Handle any errors
    print(f'Error querying user data: {e}')